### rm duplicates

In [1]:
import pandas

In [2]:
dat_file = '../data/interim/EPIv6.tab'
df = pandas.read_csv(dat_file, sep='\t')

In [5]:
df.head()

,chrom,pos,ref,alt,input_var,clin_class,pos_fam,neg_fam,hom_fam
0,1,40539750,T,C,NM_000310.3:c.904A>G,VUS,16,8529,0
1,1,40539840,T,G,NM_000310.3:c.814A>C,VUS,1,8544,0
2,1,40542543,G,T,NM_000310.3:c.769C>A,VUS,1,8544,0
3,1,40542550,T,G,NM_000310.3:c.762A>C,VUS,2,8543,0
4,1,40542572,T,C,NM_000310.3:c.740A>G,PATHOGENIC,1,8544,1


In [53]:
new_cols = ['input_var', 'clin_class', 'pos_fam', 'neg_fam', 'hom_fam']
def process_multi(r):
    rr = r[r.clin_class=='PATHOGENIC']
    if len(rr) == 1:
        return rr[new_cols]
    
    rr = r[r.clin_class=='BENIGN']
    if len(rr) == 1:
        return rr[new_cols]
    
    r.loc[:, 'pos_sum'] = r.apply(lambda row: row['hom_fam'] + row['pos_fam'], axis=1)
    max_pos = max(r['pos_sum'].values)
    rr = r[r.pos_sum==max_pos]
    if len(rr)==1:
        return rr[new_cols]
    else:
        i = 1/0
        
def choose_one(rows):
    if len(rows) == 1:
        return rows[new_cols]
    else:
        r = rows[rows.clin_class != 'VUS']
        if not r.empty:
            if len(r) == 1:
                return r[new_cols]
            else:
                return process_multi(r)
        else:
            # only vus
            process_multi(rows)

In [55]:
g_cols = ('chrom', 'pos', 'ref', 'alt')
d = df.groupby(g_cols).apply(choose_one).reset_index()

In [56]:
d[d.pos==99662302]

,chrom,pos,ref,alt,level_4,input_var,clin_class,pos_fam,neg_fam,hom_fam
3701,X,99662302,T,C,3729,NM_001184880.1:c.1294A>G,LIKELY_BENIGN,5,8399,0
